# Exercise 1

__Count the number of lines in Python for each file__

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import bz2
import time
from tqdm import tqdm


In [5]:
path_searches_bz2 = '/home/dsc/Data/challenge/searches.csv.bz2'
path_bookings_bz2 = '//home/dsc/Data/challenge/bookings.csv.bz2'

Now we extract two samples to play with them (e.g. 10000 rows)

In [6]:
!bzcat /home/dsc/Data/challenge/searches.csv.bz2 | head -10000 > searches_sample.csv
!bzcat /home/dsc/Data/challenge/bookings.csv.bz2 | head -10000 > bookings_sample.csv


bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = /home/dsc/Data/challenge/searches.csv.bz2, output file = (stdout)

bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = /home/dsc/Data/challenge/bookings.csv.bz2, output file = (stdout)


And we compress again the csv

In [7]:
!bzip2 -f ./searches_sample.csv
!bzip2 -f ./bookings_sample.csv

In [8]:
!ls -l

total 848
-rw-rw-r-- 1 dsc dsc 535893 nov 30 16:05 bookings_sample.csv.bz2
-rw-rw-r-- 1 dsc dsc  46649 nov 30 16:04 Exercise_1_and_2.ipynb
-rw-rw-r-- 1 dsc dsc  32273 nov 21 13:37 Exercise_3.ipynb
-rw-rw-r-- 1 dsc dsc     72 nov 20 17:17 README.md
-rw-rw-r-- 1 dsc dsc 244720 nov 30 16:05 searches_sample.csv.bz2


There we see that bookings_sample is approximately 0.535 MB and searches_sample 0.244 MB

In [9]:
path_searches_sample_bz2 = '/home/dsc/DS_challenge/DS_challenge/searches_sample.csv.bz2'
path_bookings_sample_bz2 = '/home/dsc/DS_challenge/DS_challenge/bookings_sample.csv.bz2'

## Shell

In [69]:
!bzcat searches_sample.csv.bz2 | tqdm --bytes | wc -l

1.74MB [00:00, 32.8MB/s]
10000


In [61]:
!bzcat /home/dsc/Data/challenge/searches.csv.bz2 | tqdm -bytes | wc -l

3.44GB [02:11, 28.1MB/s]
20390198


In [62]:
!bzcat /home/dsc/Data/challenge/bookings.csv.bz2 | tqdm -bytes | wc -l

3.95GB [02:25, 29.2MB/s]
10000011


## Python

In [91]:
searches_sample = bz2.BZ2File(path_searches_sample_bz2, mode = 'r')

In [97]:
%%time
with bz2.BZ2File(path_searches_sample_bz2, mode = 'r') as bz2file:
    k = 0
    for line in bz2file:
        k += 1
print(f'Lines: {k}')

Lines: 10000
CPU times: user 92.1 ms, sys: 0 ns, total: 92.1 ms
Wall time: 93.9 ms


This method close the file at the end

In [9]:
fileBz2.closed

True

We see it works, so we'll try to do the same with the complete files

In [98]:
%%time
with bz2.BZ2File(path_searches_bz2, mode = 'r') as bz2file:
    k = 0
    for line in bz2file:
        k += 1
print(f'Lines: {k}')

Lines: 20390199
CPU times: user 2min 20s, sys: 0 ns, total: 2min 20s
Wall time: 2min 20s


In [102]:
%%time
with bz2.BZ2File(path_bookings_bz2, mode = 'r') as bz2file:
    k = 0
    for line in bz2file:
        k += 1
print(f'Lines: {k}')

Lines: 10000011
CPU times: user 2min 17s, sys: 530 ms, total: 2min 17s
Wall time: 2min 17s


searches.csv.bz2 has __20,390,199 lines__

bookings.csv.bz2 has __10,000,011 lines__

## Pandas 

We'll create an iterable object named 'reader' which will decompress piece by piece

In [119]:
reader = pd.read_csv(path_bookings_sample_bz2, compression='bz2',
                    sep='^', iterator = True, chunksize = 10000)

A test with the sample file

In [120]:
t = time.process_time()
k = 0

for df in reader:
    k = k + len(df)
    
tfinal = time.process_time() - t
print(f'Lines {k}')
print(tfinal)

Lines 9999
0.18182607900001813


And now with the large files

In [122]:
t = time.process_time()

big_reader = pd.read_csv(path_searches_bz2, compression='bz2',
                        sep='^', iterator = True, chunksize = 10000)
k = 0

for df in big_reader:
    k = k + len(df)
    
tfinal = time.process_time() - t
print(f'Lines {k}')
print(tfinal)

Lines 20390198
221.6620747569999


In [121]:
t = time.process_time()

big_reader = pd.read_csv(path_bookings_bz2, compression='bz2',
                        sep='^', iterator = True, chunksize = 10000)
k = 0

for df in big_reader:
    k = k + len(df)
    
tfinal = time.process_time() - t
print(f'Lines {k}')
print(tfinal)

Lines 10000010
190.376313693


# Exercise 2

__Top 10 arrival airports in the world in 2013 (using the bookings file)__

* Arrival airport is the column arr_port. It is the IATA code for the airport

* To get the total number of passengers for an airport, you can sum the column pax, grouping by arr_port. Note that there is negative pax. That corresponds to cancelations. So to get the total number of passengers that have actually booked, you should sum including the negatives (that will remove the canceled bookings).

* Print the top 10 arrival airports in the standard output, including the number of passengers.

* __Bonus point__: Get the name of the city or airport corresponding to that airport (programatically, we suggest to have a look at GeoBases in Github)

* __Bonus point__: Solve this problem using pandas (instead of any other approach

In [123]:
bookings_sample = pd.read_csv('./bookings_sample.csv.bz2', compression = 'bz2', sep = '^')
bookings_sample.shape

(9999, 38)

With the sample, we should explore the data and view columns, possible NaN's, duplicates, etc

In [124]:
bookings_sample.head(10)

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,...,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546
5,2013-03-20 00:00:00,1V,US,0283592e45e392871f7e14b1e22a9aaf,2d0f8e7e1bb792625d2f34f221bc38f1,d54708298686de200f8270e7c637af5e,2013-03-20 00:00:00,5923,0,DEN,...,DENLGA,FK,W,Y,2013-04-07 10:23:00,2013-04-07 16:14:24,1,2013,3,NULL
6,2013-03-20 00:00:00,1V,US,0283592e45e392871f7e14b1e22a9aaf,2d0f8e7e1bb792625d2f34f221bc38f1,d54708298686de200f8270e7c637af5e,2013-03-20 00:00:00,5923,0,DEN,...,LGADEN,FK,K,Y,2013-04-11 11:15:00,2013-04-11 13:06:24,1,2013,3,NULL
7,2013-03-25 00:00:00,1V,JP,5af045902bd23cab579915611d99e1e0,5073861d8597467c33596bfe16f23c56,a37584d1485cb35991e4ff1a2ba92262,2013-03-25 00:00:00,8371,60,NRT,...,NRTSIN,XR,Q,Y,2013-04-14 11:05:00,2013-04-14 17:10:56,2,2013,3,NULL
8,2013-03-25 00:00:00,1V,JP,5af045902bd23cab579915611d99e1e0,5073861d8597467c33596bfe16f23c56,a37584d1485cb35991e4ff1a2ba92262,2013-03-25 00:00:00,8371,60,NRT,...,SINPEN,WS,Y,Y,2013-04-16 15:45:00,2013-04-16 17:15:29,2,2013,3,NULL
9,2013-03-25 00:00:00,1V,JP,5af045902bd23cab579915611d99e1e0,5073861d8597467c33596bfe16f23c56,a37584d1485cb35991e4ff1a2ba92262,2013-03-25 00:00:00,8371,60,NRT,...,PENBKK,EQ,Y,Y,2013-04-18 08:00:00,2013-04-18 08:53:56,2,2013,3,NULL


In [125]:
bookings_sample.columns

Index(['act_date           ', 'source', 'pos_ctry', 'pos_iata', 'pos_oid  ',
       'rloc          ', 'cre_date           ', 'duration', 'distance',
       'dep_port', 'dep_city', 'dep_ctry', 'arr_port', 'arr_city', 'arr_ctry',
       'lst_port', 'lst_city', 'lst_ctry', 'brd_port', 'brd_city', 'brd_ctry',
       'off_port', 'off_city', 'off_ctry', 'mkt_port', 'mkt_city', 'mkt_ctry',
       'intl', 'route          ', 'carrier', 'bkg_class', 'cab_class',
       'brd_time           ', 'off_time           ', 'pax', 'year', 'month',
       'oid      '],
      dtype='object')

In [12]:
len(bookings_sample['rloc          '].unique())

3638

In [14]:
list(bookings_sample.columns)

['act_date           ',
 'source',
 'pos_ctry',
 'pos_iata',
 'pos_oid  ',
 'rloc          ',
 'cre_date           ',
 'duration',
 'distance',
 'dep_port',
 'dep_city',
 'dep_ctry',
 'arr_port',
 'arr_city',
 'arr_ctry',
 'lst_port',
 'lst_city',
 'lst_ctry',
 'brd_port',
 'brd_city',
 'brd_ctry',
 'off_port',
 'off_city',
 'off_ctry',
 'mkt_port',
 'mkt_city',
 'mkt_ctry',
 'intl',
 'route          ',
 'carrier',
 'bkg_class',
 'cab_class',
 'brd_time           ',
 'off_time           ',
 'pax',
 'year',
 'month',
 'oid      ']

Remove white spaces from the columns

In [126]:
columns = list(bookings_sample.columns.values)
bookings_sample.columns = [column.strip() for column in columns]
bookings_sample.columns 

Index(['act_date', 'source', 'pos_ctry', 'pos_iata', 'pos_oid', 'rloc',
       'cre_date', 'duration', 'distance', 'dep_port', 'dep_city', 'dep_ctry',
       'arr_port', 'arr_city', 'arr_ctry', 'lst_port', 'lst_city', 'lst_ctry',
       'brd_port', 'brd_city', 'brd_ctry', 'off_port', 'off_city', 'off_ctry',
       'mkt_port', 'mkt_city', 'mkt_ctry', 'intl', 'route', 'carrier',
       'bkg_class', 'cab_class', 'brd_time', 'off_time', 'pax', 'year',
       'month', 'oid'],
      dtype='object')

In [127]:
bookings_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 38 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   act_date   9999 non-null   object
 1   source     9999 non-null   object
 2   pos_ctry   9999 non-null   object
 3   pos_iata   9999 non-null   object
 4   pos_oid    9999 non-null   object
 5   rloc       9999 non-null   object
 6   cre_date   9999 non-null   object
 7   duration   9999 non-null   int64 
 8   distance   9999 non-null   int64 
 9   dep_port   9999 non-null   object
 10  dep_city   9999 non-null   object
 11  dep_ctry   9999 non-null   object
 12  arr_port   9999 non-null   object
 13  arr_city   9999 non-null   object
 14  arr_ctry   9999 non-null   object
 15  lst_port   9999 non-null   object
 16  lst_city   9999 non-null   object
 17  lst_ctry   9999 non-null   object
 18  brd_port   9999 non-null   object
 19  brd_city   9999 non-null   object
 20  brd_ctry   9999 non-null   obj

In [138]:
bookings_sample.shape

(9999, 38)

In [140]:
bookings_sample = bookings_sample.drop_duplicates()
bookings_sample.shape

(9999, 38)

Apparently no duplicates here

In [142]:
bookings_sample[['arr_port', 'pax']]

,arr_port,pax
0,LHR,-1
1,CLT,1
2,CLT,1
3,SVO,1
4,SVO,1
...,...,...
9994,HAN,2
9995,HAN,2
9996,HAN,-2
9997,SGN,2


In [143]:
bookings_sample['pax'] = bookings_sample['pax'].dropna()

In [144]:
bookings_sample[['arr_port', 'pax']].groupby(['arr_port']).sum().sort_values('pax', ascending = False).head(10)

,pax
arr_port,
HKG,112
LGA,95
ORD,94
JFK,92
LAX,91
SFO,91
MCO,90
DCA,82
DEN,79


In [147]:
bookings_sample.groupby('arr_port').agg({"pax": "sum"}).sort_values('pax', ascending = False).head(10)

,pax
arr_port,
HKG,112
LGA,95
ORD,94
JFK,92
LAX,91
SFO,91
MCO,90
DCA,82
DEN,79


In order to manage big files, we need to learn how to chunk (i.e. taking individual pieces of information and grouping them into larger units)

Now chunk by chunk

In [10]:
i = 0
for x in pd.read_csv('bookings_sample.csv.bz2', sep = '^', chunksize = 10000):
    i += len(x)
print(i)

9999


In [14]:
#empty data frame
df = pd.DataFrame()


reader = pd.read_csv('/home/dsc/Data/challenge/bookings.csv.bz2', sep = '^', low_memory=False, chunksize = 100000)

for j, chunk in tqdm(enumerate(reader)):
    pax = chunk[['arr_port', 'pax']].copy()
    pax['pax'] = pax['pax'].dropna()
    pax_new = pax[['arr_port', 'pax']].groupby(['arr_port']).sum().reset_index()
    df = df.append(pax_new, ignore_index = True) 

50it [01:29,  1.82s/it]/home/dsc/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (7,8,27) have mixed types.Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
101it [03:04,  1.82s/it]


In [15]:
top10 = df[['arr_port', 'pax']].groupby(['arr_port']).sum().sort_values('pax', ascending = False).head(10)
top10

,pax
arr_port,
LHR,88809.0
MCO,70930.0
LAX,70530.0
LAS,69630.0
JFK,66270.0
CDG,64490.0
BKK,59460.0
MIA,58150.0
SFO,58000.0


But there is the fact that it could be full of duplicates in the large file

In [22]:
df = pd.DataFrame()


reader = pd.read_csv(path_bookings_bz2, sep = '^',
                     low_memory=False, chunksize = 100000)

for j, chunk in tqdm(enumerate(reader)):
    #drop duplicates
    df_dropped = chunk.drop_duplicates()
    #append each chunk
    df = df.append(df_dropped, ignore_index = True)
    #final drop duplicates
    bookings_clear = df.drop_duplicates()
    print(len(bookings_clear))
    if j == 15:
        break






0it [00:00, ?it/s]




1it [00:02,  2.67s/it]

100000







2it [00:06,  2.95s/it]

200000







3it [00:10,  3.41s/it]

...

1000000







14it [01:39,  9.80s/it]

1000000







15it [01:50, 10.28s/it]

1000000
1000000


We see that after reaching 1,000,000 files, the file does not increase its number of files without duplicating, so we will assume that there is no more than 1,000,000 unique rows.

In [23]:
bookings_clear.to_csv('/home/dsc/DS_challenge/DS_challenge/bookings_clear.csv.bz2',
                     sep='^', index = False, compression = 'bz2')

In [26]:
path_bookings_clear = '/home/dsc/DS_challenge/DS_challenge/bookings_clear.csv.bz2'

df = pd.DataFrame()

reader = pd.read_csv(path_bookings_clear, sep='^',
                     usecols = ['arr_port', 'pax'], low_memory=False,
                    chunksize = 100000)


In [30]:
for k, chunk in reader:
    pax = chunk[['arr_port', 'pax']].copy()
    pax['pax'] = pax['pax'].dropna()
    pax_new = pax[['arr_port', 'pax']].groupby(['arr_port'])\
    .sum().reset_index()
    
    df = df.append(pax_new, ignore_index = False)
    

In [31]:
top_10 = df[['arr_port', 'pax']].groupby(['arr_port'])\
                                      .sum()\
                                      .sort_values('pax', ascending=False)\
                                      .reset_index()\
                                      .head(10)

top_10

,arr_port,pax
0,LHR,8881
1,MCO,7093
2,LAX,7053
3,LAS,6963
4,JFK,6627
5,CDG,6449
6,BKK,5946
7,MIA,5815
8,SFO,5800
9,DXB,5559
